In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('Intent/intent_Train.csv')
df_valid = pd.read_csv('Intent/intent_Valid.csv')
df_test = pd.read_csv('Intent/intent_Test.csv')

In [29]:
df_test.head()

,utterance,intent,category,tags
0,would it be possible to cancel the order I made?,cancel_order,ORDER,BIP
1,cancelling order,cancel_order,ORDER,BK
2,I need assistance canceling the last order I h...,cancel_order,ORDER,B
3,problem with canceling the order I made,cancel_order,ORDER,B
4,I don't know how to cancel the order I made,cancel_order,ORDER,B


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6539 entries, 0 to 6538
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   utterance  6539 non-null   object
 1   intent     6539 non-null   object
 2   category   6539 non-null   object
 3   tags       6539 non-null   object
dtypes: object(4)
memory usage: 204.5+ KB


In [31]:
df.isnull().sum()

utterance    0
intent       0
category     0
tags         0
dtype: int64

In [32]:
df = df.sample(frac=1, random_state=26)#1 for 100%
df.head()

,utterance,intent,category,tags
3602,I have to correct the details on my user profile,edit_account,ACCOUNT,BLM
3068,I want assistance to check what delivery optio...,delivery_options,DELIVERY,BM
5989,I do not know what I have to do to switch to a...,switch_account,ACCOUNT,BE
3911,would it be possible to get refunds?,get_refund,REFUND,BIMP
5640,I try to set another shipping address up,set_up_shipping_address,SHIPPING_ADDRESS,B


In [33]:
df_valid['intent'].unique()

array(['cancel_order', 'change_order', 'change_shipping_address',
       'check_cancellation_fee', 'check_invoice', 'check_payment_methods',
       'check_refund_policy', 'complaint', 'contact_customer_service',
       'contact_human_agent', 'create_account', 'delete_account',
       'delivery_options', 'delivery_period', 'edit_account',
       'get_invoice', 'get_refund', 'newsletter_subscription',
       'payment_issue', 'place_order', 'recover_password',
       'registration_problems', 'review', 'set_up_shipping_address',
       'switch_account', 'track_order', 'track_refund'], dtype=object)

# Now we will load all the train test valid dataset and split them

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
df.drop(['category','tags'], axis=1, inplace=True)
df_valid.drop(['category','tags'], axis=1, inplace=True)
# df_valid.head()
df_test.drop(['category','tags'], axis=1, inplace=True)

In [36]:
X_train = df['utterance']
X_valid = df_valid['utterance']
X_test = df_test['utterance']

In [37]:
y_train = df['intent']
y_valid = df_valid['intent']
y_test = df_valid['intent']

In [48]:
X_train

3602     I have to correct the details on my user profile
3068    I want assistance to check what delivery optio...
5989    I do not know what I have to do to switch to a...
3911                 would it be possible to get refunds?
5640             I try to set another shipping address up
                              ...                        
1985                  help me contacting customer service
1456          help me check how long refunds usually take
5894                     assistance using another account
5438    is there a section to submit a review for your...
4917                          help me recover my password
Name: utterance, Length: 6539, dtype: object

# Text Vectoriztion layer 

standarizes each sample
Splits the sentence into words
recombine using ngrams
index the tokens
transform each samples to vector using indexes

In [ ]:
import tensorflow as tf

# tf.keras.layers.TextVectorization

keras.layers.preprocessing.text_vectorization.TextVectorization

In [ ]:
from concurrent.futures.process import _ThreadWakeup


# Defualt

text_Vectorizer = tf.keras.layers.TextVectorization (max_tokens= None,  # how many words in vocab
                                                     standardize="lower_and_strip_punctuation",
                                                     split= "whitespace",
                                                     ngrams= None,
                                                     output_mode="int", # can be function but we need int
                                                     output_sequence_length= None # We dont cap the output as longest sequence is unknown it will basically pad others
                                                    #  pad_to_max_tokens=True
)

To keep our data small we need the same size of number of tokens in a list

So now clculating avg length of sentences

In [ ]:
# s = 0
# for i in range(len(X_train)):
#     print(len(X_train[i].split()))
#     s = s + len(X_train[i].split()) #lENGTH OF SPLIT
# s = s/len(X_train)
# print("Average length of sequences" , s)

10
2
10
7
10
9
8
7
10
6
12
9
5
7
7
10
7
6
7
4
5
7
10
7
11
7
4
8
9
2
5
8
9
9
8
8
10
8
8
10
6
12
6
4
8
6
8
9
9
9
7
5
4
10
5
4
8
6
10
8
12
10
9
9
6
8
12
9
9
10
6
8
9
6
8
8
9
8
13
10
7
8
7
9
8
8
8
9
10
6
9
8
11
5
6
8
8
7
11
9
8
9
8
10
7
10
8
7
5
9
6
8
7
8
10
6
7
8
9
9
7
7
9
8
6
7
10
8
7
11
6
8
7
7
8
7
10
2
10
12
5
7
10
5
8
7
7
9
12
8
6
8
6
7
13
8
6
11
5
8
12
10
9
7
8
9
8
8
13
4
13
11
6
9
8
6
8
6
8
7
8
8
8
10
13
8
8
10
7
6
5
9
7
8
10
8
8
7
11
9
5
7
4
11
8
8
8
9
7
7
13
10
8
5
7
11
5
9
7
11
9
8
5
2
8
6
11
8
7
7
7
9
9
7
12
8
6
5
10
13
10
11
10
8
10
10
8
9
8
8
8
5
6
8
7
10
9
10
7
7
6
11
8
6
5
9
9
10
9
9
12
5
9
8
9
9
8
13
7
8
8
6
8
10
8
10
8
9
6
9
7
8
9
9
4
8
10
8
9
10
6
9
12
11
8
8
8
6
8
11
9
5
5
6
10
8
7
6
13
12
7
7
10
10
10
9
4
8
7
10
9
9
7
10
9
10
7
5
8
10
9
9
6
4
9
11
6
6
7
13
10
9
5
8
7
7
5
7
10
11
11
9
8
6
7
5
5
7
8
8
9
8
6
5
9
5
7
8
8
7
8
8
8
5
9
6
5
8
10
8
9
6
4
10
11
10
6
7
7
5
9
7
7
2
7
8
11
7
5
6
6
9
10
8
6
5
5
6
8
9
9
5
9
10
13
7
7
4
10
7
5
10
7
3
8
8
7
8
6
7
9
7
7
6
5
9
11
10
9
7
4

In [ ]:
# average_length = round(sum(len(i.split()) for i in X_train) / len(X_train))
# print("Average length of sequences:", average_length)


Average length of sequences: 8


In [66]:
max_vocab_length = 10000
max_length = average_length #8
#max_length = 8
text_Vectorizer = tf.keras.layers.TextVectorization (max_tokens= max_vocab_length,  # how many words in vocab
                                                    #  standardize="lower_and_strip_punctuation",
                                                    #  split= "whitespace",
                                                    #  ngrams= None,
                                                     output_mode="int", # can be function but we need int
                                                     output_sequence_length= max_length # We dont cap the output as longest sequence is unknown it will basically pad others
                                                    #  pad_to_max_tokens=True
)
 # To fit this instantance we use METHOD ADAPT

# Once adpated we can test
text_Vectorizer.adapt(X_train)

In [68]:
sample = "My name is Vedant Ranade I am Learning"
print(text_Vectorizer(sample))

tf.Tensor([ 8  1 41  1  1  2 73  1], shape=(8,), dtype=int64)


In [82]:
# We can also check details about the vocab
words = text_Vectorizer.get_vocabulary()
print(len(words))
print(f" Most common words {words[:5]}")
print(f" Least common words {words[-5:]}")

658
 Most common words ['', '[UNK]', 'i', 'to', 'the']
 Least common words ['accont', 'accesws', 'accdss', 'abuot', '7']


# Now creating Embiddings using embidding

converts integers to dense vectors of fixed size

we will set input dim: Size of Vocab

output dim : size of output embedding (10 : each token will be represented by a vector 10 long)

Length of Sequence passed

In [83]:
tf.keras.layers.Embedding(input_dim=max_vocab_length,# Input shape or Vocab
                        output_dim=128,
                        input_length=max_length# Length of Sequence
                        )